In [17]:
#Inicializando algumas librarias, se voce nao tiver alguma instale antes de continuar
import re
import json
import pandas as pd
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
from pymongo import MongoClient   
consumer_key = "Psww15xBSRVS9AIEhyQkVHHxt"
consumer_secret = "M7ynSfFLfkoJMnqigXyZzNSfIHUY7FBa72VqlYyeovdKajZZUa"
access_token = "3370825463-6jDV314VhDvhszu6Ixl1MwY2Gjr43EBn1FB4JGg"
access_token_secret = "EP3HfWYfI5zf3IGE6SH4F7YiQYKI30TL182xSXw50Q9Zd"
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
TWEETS_TO_CAPTURE = 5000

In [18]:
#Funcao para dividir uma frase em palavras
def splitTextonWords(Text, numberOfWords=1):
    if (numberOfWords > 1):
        text = Text.lstrip()
        pattern = '(?:\S+\s*){1,'+str(numberOfWords-1)+'}\S+(?!=\s*)'
        x =re.findall(pattern,text)
    elif (numberOfWords == 1):
        x = Text.split()
    else: 
        x = None
    return x

In [19]:
#Define e cria os Listener para o Twitter
def CriarListenerTwitter():
    # Criando o objeto mylistener
    global mystreammusic
    mylistenermusic = MyListenerMusic()
    global mystreambook
    mylistenerbook = MyListenerBook()
    # Criando o objeto mystream
    mystreammusic = Stream(auth, listener = mylistenermusic)
    mystreambook = Stream(auth, listener = mylistenerbook)

In [20]:
def CriarBancoMongodb():
    global colmusic
    global colbook
    global colmusicfinal
    global colbookfinal
    # Criando a conexão ao MongoDB
    client = MongoClient('localhost', 27017)
    # Criando o banco de dados twitterdb
    db = client.twitterdb
    # Criando as collecoes
    colmusic = db.tweetsmusic
    colbook = db.tweetsbook
    colmusicfinal = db.tweetsmusicfinal
    colbookfinal = db.tweetsbookfinal

In [ ]:
#Funcao para tirar duplicidade de uma lista
def duplicate(items): 
    unique = [] 
    for item in items: 
        if item not in unique: 
            unique.append(item) 
    return unique

In [ ]:
#Funcao para criar uma lista de palavras chaves de Musica que serao a chave de busca na API seleccionada
def CriarKeyMusic(data): 
    Keyword=[]
    Keyword2=[]
    Keyword3=[]
    global Keyword_Music
    Keyword_Music=[]
    s2=data.loc[data['prime_genre'] =='Music'].sort_values(by='rating_count_tot', ascending=False).head(40)[['track_name']]
    s=s2['track_name']
    for i, val in s.items():
        wordList = splitTextonWords(val, 1)
        for i in wordList:
            Keyword.append(i)
        wordList = splitTextonWords(val, 2) 
        for i in wordList:
            Keyword.append(i)
        wordList = splitTextonWords(val, 3) 
        for i in wordList:
            Keyword.append(i)    

    for i in Keyword:
        if re.search("[\u4e00-\u9FFF]", i):
            Keyword2.append(i)
        else:
            Keyword2.append(re.sub(r'[^a-zA-Z0-9-_\s]+', '', i).strip().replace("  "," "))

    Keyword3=duplicate(Keyword2)
    for i in Keyword3:
        if len(i)>4:
            if not re.sub(r'[^a-zA-Z0-9]+', '', i).islower():
                Keyword_Music.append(i.lower())

    NotKeyword_Music=['to your favorite','setlists','music player','audiobooks','karaoke -','music player','listen to', 'your favorite','listen', 'favorite', 'playlists','tempo -','with setlists','live am', 'fm radio stations', 'tempo','simple', '- live', 'fm radio','sounds', 'friends', '- music', 'and friends', 'music and friends','nicki', 'minaj', 'empire', 'nicki minaj', 'the empire','tones email', 'alerts and','- hip', 'hop albums','music radio','certified','google','google play','- mp3','free music -','top music','mixtapes for','for free','unlimited music','for youtube','music for youtube','ringtones','ringtone','ringtones for','chords','pro -','sonos','amazon','magic','free music','youtube', 'music', 'playlist', 'songs', 'player', 'iphone', 'radio', 'manager', 'search', 'email', 'create', 'audio', 'unlimited', 'ultimate', 'guitar', 'albums', 'designer', 'maker', 'discover', 'streamer', 'piano', 'karaoke', 'concerts', 'stations', 'podcasts', 'controller', 'alerts', 'tones', 'mixtapes']
    
    for ele in NotKeyword_Music: 
        if ele in Keyword_Music:
            Keyword_Music.remove(ele) 

In [ ]:
#Funcao para criar uma lista de palavras chaves de Livros que serao a chave de busca na API seleccionada
def CriarKeyBook(data):
    Keybook=[]
    Keybook2=[]
    Keybook3=[]
    global Keyword_Book
    Keyword_Book=[]
    s2=data.loc[data['prime_genre'] =='Book'].sort_values(by='rating_count_tot', ascending=False).head(50)[['track_name']]
    s=s2['track_name']
    for i, val in s.items():
        wordList = splitTextonWords(val, 2) 
        for i in wordList:
            Keybook.append(i)
        wordList = splitTextonWords(val, 3) 
        for i in wordList:
            Keybook.append(i)
        wordList = splitTextonWords(val, 4) 
        for i in wordList:
            Keybook.append(i)

    for i in Keybook:
        if re.search("[\u4e00-\u9FFF]", i):
            Keybook2.append(i)
        else:
            Keybook2.append(re.sub(r'[^a-zA-Z0-9-_\s]+', '', i).strip().replace("  "," "))

    Keybook3=duplicate(Keybook2)
    for i in Keybook3:
        if len(i)>4:
            if not re.sub(r'[^a-zA-Z0-9]+', '', i).islower():
                Keyword_Book.append(i.lower())

    NotKeyword_Book=['stories','wedding','adults','deluxe','textbooks','pumpkin','read ebooks','magazines','ebooks magazines','book for','library ebooks', 'and audiobooks','a charlie','imessage','devotional by','ebooks','track your', 'reading goals','the complete','adventures of','its the great','its the','its the great','1 the','green eggs','and ham','- read','learn','learn -','abc -','dream of','my little','hooked','hooked -','the legend','overdrive','treasury','the keeper','the square','little star','the legend of','charlie brown','sticker pack', '- read learn',
                     'pony -','the interactive', 'legend of', 'little star -','how the','christmas -','read play','the cat', 'hat -', 'the cat in', 'the hat -',
     'read learn','a space','adventure','- a space','island', '- read play', '- story','book for kids', 'batman unlimited gotham', 'or treat','one fish',
     'two fish', 'red fish', 'blue fish','blue fish -', 'oh the', 'places youll', 'most wanted','play -', 'oh the places', 'youll go -', 'oh the places youll', 'go - read']

    for ele in NotKeyword_Book: 
        if ele in Keyword_Book:
            Keyword_Book.remove(ele) 

    Keyword_Book.extend(['kindle','audible','bookshout','weirdwood','bookscanner','ibunkohd','flewn'])

In [ ]:
# Criando uma classe para capturar os stream de dados Music do Twitter e armazenar no MongoDB
musicdf = pd.DataFrame()
counter=0
class MyListenerMusic(StreamListener):
    def __init__(self, api=None):
        super(MyListenerMusic, self).__init__()
        self.num_tweets = 0
        
    def on_data(self, dados):
        global musicdf
        global counter
        self.num_tweets += 1
        tweet = json.loads(dados)
        try:
            created_at = tweet["created_at"]
        except KeyError:
            created_at = 'created_at'    
        try:
            id_str = tweet["id_str"]
        except KeyError:
            id_str = '' 
        try:
            text = tweet["text"]
        except KeyError:
            text = ''   
        obj = {"":created_at,"id_str":id_str,"text":text,}
        tweetind = colmusic.insert_one(obj).inserted_id
        
        res = [ele for ele in Keyword_Music if(ele in text)] 
        for i in res:
            musicdf = musicdf.append({'Index': counter, 'Keyword': i}, ignore_index=True)
            counter+=1

        # Stops streaming when it reaches the limit
        if self.num_tweets <= TWEETS_TO_CAPTURE:
            if self.num_tweets % 100 == 0: # just to see some progress...
                print('Number of tweets captured so far for Music: {}'.format(self.num_tweets))
            return True
        else:
            return False

In [ ]:
# Criando uma classe para capturar os stream de dados Book do Twitter e armazenar no MongoDB
bookdf = pd.DataFrame()
contador=0

class MyListenerBook(StreamListener):
    def __init__(self, api=None):        
        super(MyListenerBook, self).__init__()
        self.num_tweets = 0        
        
    def on_data(self, dados):        
        global bookdf
        global contador
        self.num_tweets += 1
        tweet = json.loads(dados)
        try:
            created_at = tweet["created_at"]
        except KeyError:
            created_at = 'created_at'    
        try:
            id_str = tweet["id_str"]
        except KeyError:
            id_str = '' 
        try:
            text = tweet["text"]
        except KeyError:
            text = ''   
        obj = {"":created_at,"id_str":id_str,"text":text,}
        tweetind = colbook.insert_one(obj).inserted_id
        
        res = [ele for ele in Keyword_Book if(ele in text)] 
        for i in res:
            bookdf = bookdf.append({'Index': contador, 'Keyword': i}, ignore_index=True)
            contador+=1
            
        # Stops streaming when it reaches the limit
        if self.num_tweets <= TWEETS_TO_CAPTURE:
            if self.num_tweets % 100 == 0: # just to see some progress...
                print('Number of tweets captured so far for Book: {}'.format(self.num_tweets))
            return True
        else:
            return False

In [ ]:
def ExtrairMusicTwitter():
    # Iniciando o filtro e gravando os tweets no MongoDB
    mystreammusic.filter(track=Keyword_Music)
    mystreammusic.disconnect()

In [ ]:
def ExtrairBookTwitter():
    # Iniciando o filtro e gravando os tweets no MongoDB
    mystreambook.filter(track=Keyword_Book)
    mystreambook.disconnect()

In [ ]:
def ExportarMusic(d):
    music=d.loc[d['prime_genre'] =='Music'].sort_values(by='rating_count_tot', ascending=False).head(40)[['id', 'track_name', 'size_bytes', 'price', 'prime_genre']]
    music['n_citacoes']=0
    s=musicdf['Keyword']
    cont=0
    for i, val in s.items():
        for x,row in music.iterrows():
            if val in row["track_name"].lower():
                cont=row["n_citacoes"]
                music.at[x, 'n_citacoes']= cont+1
    musiccsv=music.sort_values(by='n_citacoes', ascending=False).head(10)[['id', 'track_name', 'n_citacoes','size_bytes', 'price', 'prime_genre']]
    musiccsv.to_csv("Ranking_10_Music.csv", sep=';', index=False)
    
    musiccsv.to_json("Ranking_Music.json")

In [ ]:
def ExportarBook(d):
    book=d.loc[d['prime_genre'] =='Book'].sort_values(by='rating_count_tot', ascending=False).head(50)[['id', 'track_name', 'size_bytes', 'price', 'prime_genre']]
    book['n_citacoes']=0
    s=bookdf['Keyword']
    cont=0
    for i, val in s.items():
        for x,row in book.iterrows():
            if val in row["track_name"].lower():
                cont=row["n_citacoes"]
                book.at[x, 'n_citacoes']= cont+1
    bookcsv=book.sort_values(by='n_citacoes', ascending=False).head(10)[['id', 'track_name', 'n_citacoes','size_bytes', 'price', 'prime_genre']]
    bookcsv.to_csv("Ranking_10_Book.csv", sep=';', index=False)
    
    bookcsv.to_json("Ranking_Book.json")

In [ ]:
#Abrimos o arquivo AppleStore.csv
import pandas as pd
data = pd.read_csv("AppleStore.csv", 
                   encoding="utf8",
                   usecols=['id', 'track_name', 'size_bytes', 'price','rating_count_tot', 'prime_genre'])

In [ ]:
#Seleccionar a Aplicacao da categoria News com maior avaliacoes
nome_api=''
nome_api=data.loc[data['prime_genre'] =='News'].sort_values(by='rating_count_tot', ascending=False).head(1)[['track_name']].iloc[0]['track_name']

In [ ]:
#Utilizar a API seleccionada para identificar as 10 Aplicacoes mais comentadas
if nome_api=='Twitter':
    CriarKeyMusic(data)
    CriarKeyBook(data)
    CriarListenerTwitter()
    CriarBancoMongodb()
    ExtrairMusicTwitter()
    ExportarMusic(data)
    ExtrairBookTwitter()
    ExportarBook(data)
else:
    #Escrever aqui codigo para outras API     
    print("Outra API")

Number of tweets captured so far for Music: 100
Number of tweets captured so far for Music: 200
Number of tweets captured so far for Music: 300
Number of tweets captured so far for Music: 400
Number of tweets captured so far for Music: 500
Number of tweets captured so far for Music: 600
Number of tweets captured so far for Music: 700
Number of tweets captured so far for Music: 800
Number of tweets captured so far for Music: 900
Number of tweets captured so far for Music: 1000
Number of tweets captured so far for Music: 1100
Number of tweets captured so far for Music: 1200
Number of tweets captured so far for Music: 1300
Number of tweets captured so far for Music: 1400
Number of tweets captured so far for Music: 1500
Number of tweets captured so far for Music: 1600
Number of tweets captured so far for Music: 1700
Number of tweets captured so far for Music: 1800
Number of tweets captured so far for Music: 1900
Number of tweets captured so far for Music: 2000
Number of tweets captured so 